In [1]:
import pandas as pd
import numpy as np

import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from ipywidgets import IntProgress
from sklearn.metrics import classification_report
import numpy as np
tf.enable_v2_behavior()

In [2]:
adult = pd.read_csv('https://github.com/jnear/cs211-data-privacy/raw/master/homework/adult_with_pii.csv')

In [3]:
# Load data files
import numpy as np
import urllib.request
import io

url_x = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_x.npy'
url_y = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_y.npy'

with urllib.request.urlopen(url_x) as url:
    f = io.BytesIO(url.read())
X = np.load(f)

with urllib.request.urlopen(url_y) as url:
    f = io.BytesIO(url.read())
y = np.load(f)

In [4]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

x_train = X[:training_size]
x_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 36176 9044


In [5]:
y_test = np.array([1 if int(val) == 1 else 0 for val in y_test])
y_train = np.array([1 if int(val) == 1 else 0 for val in y_train])
y_test

array([0, 1, 0, ..., 0, 0, 0])

In [6]:
def tf_l2_clip(v, b):
    norm = tf.norm(v, ord=2)
    return tf.cond(norm > b, lambda: b * (v / norm), lambda: v)

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def tf_laplace_mech(v, sensitivity, epsilon):
    return tf.numpy_function(laplace_mech, [v, sensitivity, epsilon], tf.float32)

def tf_gaussian_mech(v, sensitivity, epsilon, delta):
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def tf_gaussian_mech_RDP(v, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sigma)

def tf_gaussian_mech_zCDP(v, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sigma)

In [7]:
class DPOptimizer(tf.keras.optimizers.Optimizer):
    def __init__(self, epochs, b=3.0, learning_rate=0.01, name="DPOptimizer", **kwargs):
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", learning_rate)
        self.epochs = epochs
        self.b = b
    
    def _create_slots(self, var_list):
        pass

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
        }

    
class EpsilonDeltaDPGradientDescent(DPOptimizer):
    def __init__(self, epochs, epsilon, delta, b=3.0, learning_rate=0.01, name="EpsilonDeltaDPGradientDescent", **kwargs):
        DPOptimizer.__init__(self, epochs, b=b, learning_rate=learning_rate, name=name, **kwargs)        
        self.epsilon = epsilon
        self.delta = delta

    @tf.function
    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        
        epsilon_i = self.epsilon / self.epochs
        delta_i = self.delta / self.epochs
        
        clipped_grad = tf.math.reduce_mean(tf_l2_clip(grad, self.b), axis=0)
        #clipped_grad = tf.numpy_function(lambda x: np.mean(x, axis=0), [tf_l2_clip(grad, self.b)], tf.float32)
        #clipped_grad = tf.math.reduce_mean(tf.clip_by_norm(grad, self.b), axis=0)
        #clipped_grad = tf_l2_clip(grad, self.b)
        new_var_m = var - tf_gaussian_mech(clipped_grad, self.b/len(x_train), epsilon_i, delta_i) * lr_t
        #new_var_m = var - grad * lr_t
        
        new_var = new_var_m
        var.assign(new_var)

In [8]:
model = models.Sequential([
    layers.Dense(1024, input_shape=(X.shape[1],)),
    layers.Activation('relu'),
    layers.Dense(1024),
    layers.Activation('relu'),
    layers.Dense(1024),
    layers.Activation('relu'),
    layers.Dense(1),
    layers.Activation('sigmoid')
])

model.compile(optimizer=EpsilonDeltaDPGradientDescent(epochs=3, epsilon=0.1, delta=1e-5), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(x=x_train,y=y_train, epochs=3, batch_size=64)
accuracy = model.evaluate(x_test, y_test)
accuracy

Epoch 1/3
566/566 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.7506
Epoch 2/3
566/566 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.7506
Epoch 3/3
283/283 [==============================] - 0s 636us/step - loss: nan - accuracy: 0.7585


[nan, 0.7585139274597168]

- joe's algorithm unmodified in example sort of falls off a cliff too
- reduce mean works on adult dataset
- reduce mean doesn't work on flattened mnist, so it's probably not dimensions issue